# Ranked Choice Voting

This is my space to program an application that supports ranked-choice voting. In this system, participants are given a list of options. The participant assigns a number to indicate order of preference, with the lower number being preferred over the higher number.

To calculate the winners, all ballots are considered. The first round considers all of the first choices. If the most popular option receives more than 50% of the votes, that option is the winner. If no option receives 50% of the votes, the least favorite option gets removed from consideration, and a new tally is taken. The run-off process continues until one option receives more than 50% of the votes.

## Example

Say there is a vote about what to order for a group lunch.

Someone who enjoys meat and vegetables might vote this way:

* Roasted chicken - 3
* Broccoli - 2
* Sausage links - 1
* Beef brisket - 5
* Artichoke salad - 4

Participants are allowed to abstain from ranking options that do not appeal at all. For example, when selecting food to order, a meat-lover might submit the following rankings:

* Roasted chicken - 3
* Broccoli
* Sausage links - 2
* Beef brisket - 1
* Artichoke salad

The participant is declaring that, if all of the ranked options are eliminated during the run-offs, he does not have a preference for the remaining options.


## Algorithm for Instant-runoff voting

From perplexity, which references ballotpedia.org

In instant-runoff voting (IRV), if there is no winner in the first round and there are multiple candidates with the fewest votes, the process typically involves eliminating all candidates with the fewest votes simultaneously. Here’s how it works:

1. Initial Count: All first-choice votes are tallied.
2. Majority Check: If a candidate receives more than 50% of the first-choice votes, they win.
3. Elimination of Candidates: If no candidate has a majority, the candidate(s) with the fewest votes are eliminated. If multiple candidates are tied for the fewest votes, all of them are eliminated simultaneously.
Redistribution of Votes: Votes from the eliminated candidates are redistributed to the next preferred candidate on each ballot.
4. Recount: The votes are recounted with the updated tallies.
5. Repeat: Steps 3-5 are repeated until a candidate receives more than 50% of the votes and is declared the winner.

This approach ensures that the process continues smoothly even when multiple candidates have the same number of lowest votes, by eliminating them all at once and redistributing their votes accordingly.

In [1]:
print("This is my first line of python in this notebook.")

This is my first line of python in this notebook.


In [15]:
# we need some data...what data do we need?
elections = [
    { 'name' : 'Best Superhero', 'code' : 'best_hero', 'description' : 'Decide who is the best superhero?' }
]

choices = [
    { 'name' : 'Batman', 'code' : 'batman', 'description' : 'Defender of Gotham', 'election_code' : 'best_hero' },
    { 'name' : 'Spiderman', 'code' : 'spiderman', 'description' : 'Your local neighborhood hero', 'election_code' : 'best_hero' },
    { 'name' : 'Black Panther', 'code' : 'blackpanther', 'description' : 'The pride of Wakanda', 'election_code' : 'best_hero' }
]

participants = [
    { 'name' : 'Annie', 'ID' : 'a10000', 'email' : 'annie@somedomain.aaa', 'verified' : True },
    { 'name' : 'Bart', 'ID' : 'a10001', 'email' : 'bart@somedomain.aaa', 'verified' : True },
    { 'name' : 'Cindy', 'ID' : 'a10002', 'email' : 'cindy@somedomain.aaa', 'verified' : True },
    { 'name' : 'Duddly', 'ID' : 'a10003', 'email' : 'duddly@somedomain.aaa', 'verified' : True },
    { 'name' : 'Earnest', 'ID' : 'a10004', 'email' : 'earnest@somedomain.aaa', 'verified' : True }
]

ballots = [
    { 
        'election' : 'best_hero', 
        'participant' : 'a10000',
        'ranking' : [
            {'batman' : 3},
            {'spiderman' : 1},
            {'blackpanther' : 2}
        ]
    },
    {
        'election' : 'best_hero', 
        'participant' : 'a10001',
        'ranking' : [
            {'batman' : 3},
            {'spiderman' : 2},
            {'blackpanther' : 1}
        ]
    },
    {
        'election' : 'best_hero', 
        'participant' : 'a10002',
        'ranking' : [
            {'batman' : 1},
            {'blackpanther' : 2}
        ]
    },
    {
        'election' : 'best_hero', 
        'participant' : 'a10003',
        'ranking' : [
            {'batman' : 1},
            {'spiderman' : 3},
            {'blackpanther' : 2}
        ]
    },
    {
        'election' : 'best_hero', 
        'participant' : 'a10004',
        'ranking' : [
            {'batman' : 2},
            {'spiderman' : 1},
            {'blackpanther' : 3}
        ]
    }
]


In [16]:
def sort_heroes_by_ranking(ranking_list):
    # Create a list of tuples (hero, rank)
    hero_rank_pairs = [(list(item.keys())[0], list(item.values())[0]) for item in ranking_list]
    
    # Sort the list based on the rank (second element of each tuple)
    sorted_pairs = sorted(hero_rank_pairs, key=lambda x: x[1])
    
    # Extract only the hero names from the sorted list
    sorted_heroes = [hero for hero, _ in sorted_pairs]
    
    return sorted_heroes

ranking = [
    {'batman': 3},
    {'spiderman': 1},
    {'blackpanther': 2}
]
result = sort_heroes_by_ranking(ranking)
print(result)

['spiderman', 'blackpanther', 'batman']


In [29]:
class Candidate:
    def __init__(self, name):
        self.name = name
        self.eliminated = False

class Ballot:
    def __init__(self, rankings):
        self.rankings = rankings  # List of Candidate objects in order of preference

class Election:
    def __init__(self, candidates):
        self.candidates = {candidate.name: candidate for candidate in candidates}
        self.ballots = []

    def add_ballot(self, rankings):
        ballot = Ballot([self.candidates[name] for name in rankings])
        self.ballots.append(ballot)

    def count_first_choices(self):
        counts = {candidate: 0 for candidate in self.candidates.values()}
        for ballot in self.ballots:
            for candidate in ballot.rankings:
                if not candidate.eliminated:
                    counts[candidate] += 1
                    break
        return counts

    def eliminate_candidate(self, candidate):
        candidate.eliminated = True

    def run_election(self):
        while True:
            counts = self.count_first_choices()
            total_votes = sum(counts.values())
            
            for candidate, votes in counts.items():
                if votes > total_votes / 2:
                    return candidate.name  # Winner found
            
            # No winner yet, eliminate the candidate with the least votes
            loser = min(counts, key=counts.get)
            print(loser.name + ' is the loser in this round')
            self.eliminate_candidate(loser)
            
            if len([c for c in self.candidates.values() if not c.eliminated]) == 1:
                return next(c.name for c in self.candidates.values() if not c.eliminated)

# Example usage
candidates = [Candidate("Alice"), Candidate("Bob"), Candidate("Charlie")]
election = Election(candidates)

# Add sample ballots
election.add_ballot(["Alice", "Charlie", "Bob"])
election.add_ballot(["Bob", "Alice", "Charlie"])
election.add_ballot(["Charlie", "Alice", "Bob"])

winner = election.run_election()
print(f"The winner is: {winner}")


Bob is the loser in this round
The winner is: Alice


This is a nice solution generated by Claude 3.5. It's got reasonable objects to support the election and find a winner when there is a clear loser in every round.

However, it does not handle a few things:
* There is no history to see how each candidate did in each round.
* In the case of a tie for the worst candidate, the candidate that appears first in the list is eliminated. So sort order matters, which seems unfair. In a better system, all candidates tied for worst get eliminated.
* In the extreme case where all candidates tie in every round, the election result should be "no winner." This might even be true if candidates are eliminated, and only the final round and all tie breakers come out as a tie.

So I asked Claude 3.5 to track rounds in order to help settle tiebreakers. Here's the modified code:


In [31]:
from collections import defaultdict

class Candidate:
    def __init__(self, name):
        self.name = name
        self.eliminated = False

class Ballot:
    def __init__(self, rankings):
        self.rankings = rankings  # List of Candidate objects in order of preference

class Round:
    def __init__(self, vote_counts, eliminated_candidate=None):
        self.vote_counts = vote_counts
        self.eliminated_candidate = eliminated_candidate

class Election:
    def __init__(self, candidates):
        self.candidates = {candidate.name: candidate for candidate in candidates}
        self.ballots = []
        self.rounds = []

    def add_ballot(self, rankings):
        ballot = Ballot([self.candidates[name] for name in rankings])
        self.ballots.append(ballot)

    def count_votes(self):
        counts = defaultdict(int)
        for ballot in self.ballots:
            for candidate in ballot.rankings:
                if not candidate.eliminated:
                    counts[candidate] += 1
                    break
        return counts

    def eliminate_candidate(self, candidate):
        candidate.eliminated = True

    def resolve_tie(self, tied_candidates):
        # Check previous rounds in reverse order
        for round in reversed(self.rounds):
            min_votes = min(round.vote_counts[c] for c in tied_candidates)
            candidates_with_min_votes = [c for c in tied_candidates if round.vote_counts[c] == min_votes]
            if len(candidates_with_min_votes) < len(tied_candidates):
                return candidates_with_min_votes[0]
        
        # If still tied after checking all rounds, choose arbitrarily
        return tied_candidates[0]

    def run_election(self):
        while True:
            vote_counts = self.count_votes()
            total_votes = sum(vote_counts.values())
            
            # Record this round
            self.rounds.append(Round(vote_counts.copy()))
            
            for candidate, votes in vote_counts.items():
                if votes > total_votes / 2:
                    return candidate.name  # Winner found
            
            # Find candidate(s) with the least votes
            min_votes = min(vote_counts.values())
            candidates_to_eliminate = [c for c, v in vote_counts.items() if v == min_votes]
            
            if len(candidates_to_eliminate) > 1:
                # Tie for last place, use tie-breaking logic
                to_eliminate = self.resolve_tie(candidates_to_eliminate)
            else:
                to_eliminate = candidates_to_eliminate[0]
            
            self.eliminate_candidate(to_eliminate)
            self.rounds[-1].eliminated_candidate = to_eliminate
            
            if len([c for c in self.candidates.values() if not c.eliminated]) == 1:
                return next(c.name for c in self.candidates.values() if not c.eliminated)

    def print_results(self):
        for i, round in enumerate(self.rounds, 1):
            print(f"Round {i}:")
            for candidate, votes in round.vote_counts.items():
                print(f"  {candidate.name}: {votes}")
            if round.eliminated_candidate:
                print(f"  Eliminated: {round.eliminated_candidate.name}")
            print()

# Example usage
candidates = [Candidate("Alice"), Candidate("Bob"), Candidate("Charlie"), Candidate("David"), Candidate("Elton")]
election = Election(candidates)

# Add sample ballots
election.add_ballot(["Alice", "Bob", "Charlie", "David"])
election.add_ballot(["Bob", "Alice", "Charlie", "David"])
election.add_ballot(["Charlie", "David", "Alice", "Bob"])
election.add_ballot(["David", "Charlie", "Bob", "Alice"])
election.add_ballot(["Elton", "Charlie", "Bob", "David"])

winner = election.run_election()
print(f"The winner is: {winner}")
election.print_results()


The winner is: Bob
Round 1:
  Alice: 1
  Bob: 1
  Charlie: 1
  David: 1
  Elton: 1
  Eliminated: Alice

Round 2:
  Bob: 2
  Charlie: 1
  David: 1
  Elton: 1
  Eliminated: Charlie

Round 3:
  Bob: 2
  David: 2
  Elton: 1
  Eliminated: Elton

Round 4:
  Bob: 3
  David: 2



Next, the first tie breaker rule is:

1) If all candidates have the same number of votes in a given round, that round is marked as a tie. Do the next runoff using the next-ranked candidates.

Claude updated the code to:

In [ ]:
from collections import defaultdict

class Candidate:
    def __init__(self, name):
        self.name = name
        self.eliminated = False

class Ballot:
    def __init__(self, rankings):
        self.rankings = rankings  # List of Candidate objects in order of preference
        self.current_rank = 0

    def get_next_choice(self):
        while self.current_rank < len(self.rankings):
            candidate = self.rankings[self.current_rank]
            self.current_rank += 1
            if not candidate.eliminated:
                return candidate
        return None

class Round:
    def __init__(self, vote_counts, eliminated_candidate=None, is_tie=False):
        self.vote_counts = vote_counts
        self.eliminated_candidate = eliminated_candidate
        self.is_tie = is_tie

class Election:
    def __init__(self, candidates):
        self.candidates = {candidate.name: candidate for candidate in candidates}
        self.ballots = []
        self.rounds = []

    def add_ballot(self, rankings):
        ballot = Ballot([self.candidates[name] for name in rankings])
        self.ballots.append(ballot)

    def count_votes(self):
        counts = defaultdict(int)
        for ballot in self.ballots:
            choice = ballot.get_next_choice()
            if choice:
                counts[choice] += 1
        return counts

    def eliminate_candidate(self, candidate):
        candidate.eliminated = True

    def is_tie(self, vote_counts):
        return len(set(vote_counts.values())) == 1 and len(vote_counts) > 1

    def run_election(self):
        # FIXME: turning this off because it going into an infinite loop when all rounds are meant to end in a tie
        while False: 
            vote_counts = self.count_votes()
            total_votes = sum(vote_counts.values())
            
            is_tie_round = self.is_tie(vote_counts)
            
            # Record this round
            self.rounds.append(Round(vote_counts.copy(), is_tie=is_tie_round))
            
            if is_tie_round:
                print(f"Tie detected in round {len(self.rounds)}. Moving to next choices.")
                # Reset current_rank for all ballots to consider next choices
                for ballot in self.ballots:
                    ballot.current_rank = 0
                continue
            
            for candidate, votes in vote_counts.items():
                if votes > total_votes / 2:
                    return candidate.name  # Winner found
            
            # Find candidate(s) with the least votes
            min_votes = min(vote_counts.values())
            candidates_to_eliminate = [c for c, v in vote_counts.items() if v == min_votes]
            
            to_eliminate = candidates_to_eliminate[0]  # In case of tie at this point, eliminate arbitrarily
            
            self.eliminate_candidate(to_eliminate)
            self.rounds[-1].eliminated_candidate = to_eliminate
            
            if len([c for c in self.candidates.values() if not c.eliminated]) == 1:
                return next(c.name for c in self.candidates.values() if not c.eliminated)

    def print_results(self):
        for i, round in enumerate(self.rounds, 1):
            print(f"Round {i}:")
            for candidate, votes in round.vote_counts.items():
                print(f"  {candidate.name}: {votes}")
            if round.is_tie:
                print("  This round was a tie. Moving to next choices.")
            elif round.eliminated_candidate:
                print(f"  Eliminated: {round.eliminated_candidate.name}")
            print()

# Example usage
candidates = [Candidate("Alice"), Candidate("Bob"), Candidate("Charlie"), Candidate("David")]
election = Election(candidates)

# Add sample ballots
election.add_ballot(["Alice", "Bob", "Charlie", "David"])
election.add_ballot(["Bob", "Alice", "Charlie", "David"])
election.add_ballot(["Charlie", "David", "Alice", "Bob"])
election.add_ballot(["David", "Charlie", "Bob", "Alice"])

winner = election.run_election()
print(f"The winner is: {winner}")
election.print_results()

This code is groovy, except in the case where each round is a tie. The code goes into an infinite loop.

I explained to Claude:

1) We should stop runoffs once all choices on all ballots have been exhausted.
2) In the case that every round is a tie, declare the result as "No winner."

The next iteration looks like:

In [1]:
from collections import defaultdict

class Candidate:
    def __init__(self, name):
        self.name = name
        self.eliminated = False

class Ballot:
    def __init__(self, rankings):
        self.rankings = rankings  # List of Candidate objects in order of preference
        self.current_rank = 0

    def get_next_choice(self):
        while self.current_rank < len(self.rankings):
            candidate = self.rankings[self.current_rank]
            self.current_rank += 1
            if not candidate.eliminated:
                return candidate
        return None

    def has_more_choices(self):
        return self.current_rank < len(self.rankings)

class Round:
    def __init__(self, vote_counts, eliminated_candidate=None, is_tie=False):
        self.vote_counts = vote_counts
        self.eliminated_candidate = eliminated_candidate
        self.is_tie = is_tie

class Election:
    def __init__(self, candidates):
        self.candidates = {candidate.name: candidate for candidate in candidates}
        self.ballots = []
        self.rounds = []

    def add_ballot(self, rankings):
        ballot = Ballot([self.candidates[name] for name in rankings])
        self.ballots.append(ballot)

    def count_votes(self):
        counts = defaultdict(int)
        for ballot in self.ballots:
            choice = ballot.get_next_choice()
            if choice:
                counts[choice] += 1
        return counts

    def eliminate_candidate(self, candidate):
        candidate.eliminated = True

    def is_tie(self, vote_counts):
        return len(set(vote_counts.values())) == 1 and len(vote_counts) > 1

    def any_ballots_have_more_choices(self):
        return any(ballot.has_more_choices() for ballot in self.ballots)

    def run_election(self):
        # FIXME: turning this off because it going into an infinite loop when all rounds are meant to end in a tie
        while False: 
            vote_counts = self.count_votes()
            
            if not vote_counts:
                # No more valid votes, end the election
                self.rounds.append(Round({}, is_tie=True))
                return "No winner"

            total_votes = sum(vote_counts.values())
            
            is_tie_round = self.is_tie(vote_counts)
            
            # Record this round
            self.rounds.append(Round(vote_counts.copy(), is_tie=is_tie_round))
            
            if is_tie_round:
                if not self.any_ballots_have_more_choices():
                    return "No winner"  # Tie in final round
                print(f"Tie detected in round {len(self.rounds)}. Moving to next choices.")
                # Reset current_rank for all ballots to consider next choices
                for ballot in self.ballots:
                    ballot.current_rank = 0
                continue
            
            for candidate, votes in vote_counts.items():
                if votes > total_votes / 2:
                    return candidate.name  # Winner found
            
            # Find candidate(s) with the least votes
            min_votes = min(vote_counts.values())
            candidates_to_eliminate = [c for c, v in vote_counts.items() if v == min_votes]
            
            to_eliminate = candidates_to_eliminate[0]  # In case of tie at this point, eliminate arbitrarily
            
            self.eliminate_candidate(to_eliminate)
            self.rounds[-1].eliminated_candidate = to_eliminate
            
            if len([c for c in self.candidates.values() if not c.eliminated]) == 1:
                return next(c.name for c in self.candidates.values() if not c.eliminated)

    def print_results(self):
        for i, round in enumerate(self.rounds, 1):
            print(f"Round {i}:")
            for candidate, votes in round.vote_counts.items():
                print(f"  {candidate.name}: {votes}")
            if round.is_tie:
                print("  This round was a tie.")
                if i == len(self.rounds):
                    print("  No more choices available. Election ended in a tie.")
            elif round.eliminated_candidate:
                print(f"  Eliminated: {round.eliminated_candidate.name}")
            print()

# Example usage
candidates = [Candidate("Alice"), Candidate("Bob"), Candidate("Charlie"), Candidate("David")]
election = Election(candidates)

# Add sample ballots that will result in a tie every round
election.add_ballot(["Alice", "Bob", "Charlie", "David"])
election.add_ballot(["Bob", "Charlie", "David", "Alice"])
election.add_ballot(["Charlie", "David", "Alice", "Bob"])
election.add_ballot(["David", "Alice", "Bob", "Charlie"])

winner = election.run_election()
print(f"The result is: {winner}")
election.print_results()

The result is: None


That still has an infinite loop, which is obvious from running it as is. This demonstrates the importance of testing. The code may seem to make sense, but you need to make sure it works under different conditions. We might call these use cases or edge conditions ("extreme" use cases we didn't think of at first).

This time I told Claude that the infinite loop problem is still there. I asked if it could see the problem. Claude apologized for the oversite and modified the solution.

In [4]:
class Election:
    def __init__(self, candidates):
        self.candidates = {candidate.name: candidate for candidate in candidates}
        self.ballots = []
        self.rounds = []

    def add_ballot(self, rankings):
        ballot = Ballot([self.candidates[name] for name in rankings])
        self.ballots.append(ballot)

    def count_votes(self):
        counts = defaultdict(int)
        for ballot in self.ballots:
            choice = ballot.get_next_choice()
            if choice:
                counts[choice] += 1
        return counts

    def eliminate_candidate(self, candidate):
        candidate.eliminated = True

    def is_tie(self, vote_counts):
        return len(set(vote_counts.values())) == 1 and len(vote_counts) > 1

    def any_ballots_have_more_choices(self):
        return any(ballot.has_more_choices() for ballot in self.ballots)

    # NOTE: only this method changed from the previous iteration
    def run_election(self):
        while True:
            vote_counts = self.count_votes()
            
            if not vote_counts:
                # No more valid votes, end the election
                self.rounds.append(Round({}, is_tie=True))
                return "No winner"

            total_votes = sum(vote_counts.values())
            
            is_tie_round = self.is_tie(vote_counts)
            
            # Record this round
            self.rounds.append(Round(vote_counts.copy(), is_tie=is_tie_round))
            
            if is_tie_round:
                if not self.any_ballots_have_more_choices():
                    return "No winner"  # Tie in final round
                print(f"Tie detected in round {len(self.rounds)}. Moving to next choices.")
                continue  # Move to next round without resetting ballots
            
            for candidate, votes in vote_counts.items():
                if votes > total_votes / 2:
                    return candidate.name  # Winner found
            
            # Find candidate(s) with the least votes
            min_votes = min(vote_counts.values())
            candidates_to_eliminate = [c for c, v in vote_counts.items() if v == min_votes]
            
            to_eliminate = candidates_to_eliminate[0]  # In case of tie at this point, eliminate arbitrarily
            
            self.eliminate_candidate(to_eliminate)
            self.rounds[-1].eliminated_candidate = to_eliminate
            
            if len([c for c in self.candidates.values() if not c.eliminated]) == 1:
                return next(c.name for c in self.candidates.values() if not c.eliminated)

    def print_results(self):
        for i, round in enumerate(self.rounds, 1):
            print(f"Round {i}:")
            for candidate, votes in round.vote_counts.items():
                print(f"  {candidate.name}: {votes}")
            if round.is_tie:
                print("  This round was a tie.")
                if i == len(self.rounds):
                    print("  No more choices available. Election ended in a tie.")
            elif round.eliminated_candidate:
                print(f"  Eliminated: {round.eliminated_candidate.name}")
            print()

# Example usage
candidates = [Candidate("Alice"), Candidate("Bob"), Candidate("Charlie"), Candidate("David")]
election = Election(candidates)

# Add sample ballots that will result in a tie every round
election.add_ballot(["Alice", "Bob", "Charlie", "David"])
election.add_ballot(["Bob", "Charlie", "David", "Alice"])
election.add_ballot(["Charlie", "David", "Alice", "Bob"])
election.add_ballot(["David", "Alice", "Bob", "Charlie"])

winner = election.run_election()
print(f"The result is: {winner}")
election.print_results()

Tie detected in round 1. Moving to next choices.
Tie detected in round 2. Moving to next choices.
Tie detected in round 3. Moving to next choices.
The result is: No winner
Round 1:
  Alice: 1
  Bob: 1
  Charlie: 1
  David: 1
  This round was a tie.

Round 2:
  Bob: 1
  Charlie: 1
  David: 1
  Alice: 1
  This round was a tie.

Round 3:
  Charlie: 1
  David: 1
  Alice: 1
  Bob: 1
  This round was a tie.

Round 4:
  David: 1
  Alice: 1
  Bob: 1
  Charlie: 1
  This round was a tie.
  No more choices available. Election ended in a tie.



This solution works in the case of an absolute tie, where every candidate receives the same number of votes in each round, so no candidates are eliminated and no winner is found. Well done, Claude.

Now I want to be able to easily run test cases. I'll code this part myself to get some practice with Python.

In [13]:
def run_test(test_candidate_names, test_ballots, expected_winner):
    
    candidates = [Candidate(candidate_name) for candidate_name in test_candidate_names]
    election = Election(candidates)
    
    # Add sample ballots that will result in a tie every round
    for ballot_ranking in test_ballots:
        election.add_ballot(ballot_ranking)
    
    winner = election.run_election()

    print(f"The result is: {winner}")
    election.print_results()
    
    passing_test = expected_winner == winner
    print(f"The test { 'Passed' if passing_test else 'Failed' }\n************\n\n\n")
    
    return passing_test

candidates = ["Alice", "Bob", "Charlie", "David"]
ballots = [
    ["Alice", "Bob", "Charlie", "David"],
    ["Bob", "Charlie", "David", "Alice"],
    ["Charlie", "David", "Alice", "Bob"],
    ["David", "Alice", "Bob", "Charlie"]
]
run_test(candidates, ballots, 'No winner')


Tie detected in round 1. Moving to next choices.
Tie detected in round 2. Moving to next choices.
Tie detected in round 3. Moving to next choices.
The result is: No winner
Round 1:
  Alice: 1
  Bob: 1
  Charlie: 1
  David: 1
  This round was a tie.

Round 2:
  Bob: 1
  Charlie: 1
  David: 1
  Alice: 1
  This round was a tie.

Round 3:
  Charlie: 1
  David: 1
  Alice: 1
  Bob: 1
  This round was a tie.

Round 4:
  David: 1
  Alice: 1
  Bob: 1
  Charlie: 1
  This round was a tie.
  No more choices available. Election ended in a tie.

The test Passed
************





True

Excellent. Now let's test more cases.

In [17]:
def test_all_ways_tie():
    candidates = ["Alice", "Dave", "Dilbert", "Wally"]
    ballots = [
        ["Alice", "Dave", "Dilbert", "Wally"],
        ["Dave", "Dilbert", "Wally", "Alice"],
        ["Dilbert", "Wally", "Alice", "Dave"],
        ["Wally", "Alice", "Dave", "Dilbert"]
    ]
    return run_test(candidates, ballots, 'No winner')

def test_we_love_alice():
    candidates = ["Alice", "Dave", "Dilbert", "Wally"]
    ballots = [
        ["Alice", "Dave", "Dilbert", "Wally"],
        ["Alice", "Dilbert", "Wally", "Dave"],
        ["Alice", "Wally", "Dilbert", "Dave"],
        ["Alice", "Wally", "Dave", "Dilbert"]
    ]
    return run_test(candidates, ballots, 'Alice')

def test_dilbert_wally_death_match():
    candidates = ["Alice", "Dave", "Dilbert", "Wally"]
    ballots = [
        ["Dilbert", "Dave", "Alice", "Wally"],
        ["Dilbert", "Alice", "Dave", "Wally"],
        ["Wally", "Dave", "Alice", "Dilbert"],
        ["Wally", "Alice", "Dave", "Dilbert"]
    ]
    return run_test(candidates, ballots, 'No winner')


test_all_ways_tie()
test_we_love_alice()
test_dilbert_wally_death_match()


Tie detected in round 1. Moving to next choices.
Tie detected in round 2. Moving to next choices.
Tie detected in round 3. Moving to next choices.
The result is: No winner
Round 1:
  Alice: 1
  Dave: 1
  Dilbert: 1
  Wally: 1
  This round was a tie.

Round 2:
  Dave: 1
  Dilbert: 1
  Wally: 1
  Alice: 1
  This round was a tie.

Round 3:
  Dilbert: 1
  Wally: 1
  Alice: 1
  Dave: 1
  This round was a tie.

Round 4:
  Wally: 1
  Alice: 1
  Dave: 1
  Dilbert: 1
  This round was a tie.
  No more choices available. Election ended in a tie.

The test Passed
************



The result is: Alice
Round 1:
  Alice: 4

The test Passed
************



Tie detected in round 1. Moving to next choices.
Tie detected in round 2. Moving to next choices.
Tie detected in round 3. Moving to next choices.
The result is: No winner
Round 1:
  Dilbert: 2
  Wally: 2
  This round was a tie.

Round 2:
  Dave: 2
  Alice: 2
  This round was a tie.

Round 3:
  Alice: 2
  Dave: 2
  This round was a tie.

Round 4:
  W

True

In [ ]:
# Test cases
def run_all_tests():
    tests = [
        {
            "name": "Simple Majority Winner",
            "candidates": ["Alice", "Bob", "Charlie"],
            "ballots": [
                ["Alice", "Bob", "Charlie"],
                ["Alice", "Charlie", "Bob"],
                ["Bob", "Alice", "Charlie"],
                ["Alice", "Bob", "Charlie"]
            ],
            "expected": "Alice"
        },
        {
            "name": "Runoff Required",
            "candidates": ["Alice", "Bob", "Charlie"],
            "ballots": [
                ["Alice", "Bob", "Charlie"],
                ["Bob", "Alice", "Charlie"],
                ["Charlie", "Alice", "Bob"],
                ["Charlie", "Bob", "Alice"]
            ],
            "expected": "Alice"
        },
        {
            "name": "Tie Result",
            "candidates": ["Alice", "Bob", "Charlie", "David"],
            "ballots": [
                ["Alice", "Bob", "Charlie", "David"],
                ["Bob", "Charlie", "David", "Alice"],
                ["Charlie", "David", "Alice", "Bob"],
                ["David", "Alice", "Bob", "Charlie"]
            ],
            "expected": "No winner"
        }
    ]

    for test in tests:
        print(f"Running test: {test['name']}")
        result = run_test(test["candidates"], test["ballots"], test["expected"])
        if not result:
            print(f"Test failed: {test['name']}")
            return False
    
    print("All tests passed successfully!")
    return True

# Run all tests
run_all_tests()